<a href="https://colab.research.google.com/github/asumansaree/All-HackerRank-C-Challanges/blob/main/merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import random
import json
import pickle # for serialization
import numpy as np
import codecs
import io

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer # for stem find

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

from distutils.command.clean import clean
from unittest import result # for serialization

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer # for stem find

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer("tr") # call the constructor

# reads data.json as text and loads func returns it as json object (in dict structure)
intents = json.load(codecs.open('tag_pattern_response_wordGroups.json', 'r', 'utf-8-sig'))

# create empty lists
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# iterate over the intents
# nested for loop is necessary for key's inside dict
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern) # tokenize splits the sentence into words
        words.extend(word_list) # extend = take the content and append it to the list
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
          classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words)) # set eliminate the duplicates, sorted trans it backs to list

classes = sorted(set(classes))

# to save them into a file
pickle.dump(words, open('words_demo.pkl', 'wb'))
pickle.dump(classes, open('classes_demo.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer =sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel_demo.h5', hist)
print("Done")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


TypeError: ignored

In [ ]:
raç

In [54]:
lemmatizer = WordNetLemmatizer()

with io.open('tag_pattern_response_wordGroups.json', 'r', encoding='utf-8-sig') as f:
  intents = json.loads(f.read())

#intents = json.loads(open('data.json').read())

words = pickle.load(open('words_demo.pkl', 'rb'))
classes = pickle.load(open('classes_demo.pkl', 'rb'))
model = load_model('chatbotmodel_demo.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_TRESHOLD = 0.05
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_TRESHOLD]
    
    results.sort(key=lambda x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    print("ints:\n", intents_list)
    # print("prob:\n", prob)
    # print("list_of_intents:\n", list_of_intents)
    
    # best_probability = intents_list[0]['probability']
    # best_match_response = list_of_intents['responses']
    # for i in list_of_intents:
    #     if i['tag'] == tag:
    #         #result = random.choice(i['responses'])
    #         if intents_list[i]['probability'] > best_probability:
    #           best_probability = intents_list[i]['probability']
    #           best_match_response = i['responses']
    # print(best_probability)
    # return best_match_response
    best_prob = intents_list[0]['probability']
    best_i = 0
    for i in range(len(intents_list)):
      if intents_list[i]['probability'] > best_prob:
          best_prob = intents_list[i]['probability']
          best_i = i
    best_matched_tag = intents_list[best_i]['intent']
    for i in list_of_intents:
        if i['tag'] == best_matched_tag:
          return i['responses']
    return -1

print("GO! Bot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

GO! Bot is running!
SES ÇOK YÜKSEK SES ÖLÇÜMÜ 


IndexError: ignored